In [2]:
!pip install pyspark

     |████████████████████████████████| 281.4 MB 31 kB/s 
     |████████████████████████████████| 198 kB 44.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=6f6799e1b3103b99c98add133bc9420945eea217062eccad32066f42aa6ded09
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [3]:
from pyspark.sql import SparkSession

In [4]:
session=SparkSession.builder.appName("Decision").getOrCreate()

In [37]:
data=session.read.csv("letter_recognition.csv",header=True,inferSchema=True)

In [38]:
data.show(20)

+------+----+----+-----+----+-----+----+----+-----+-----+-----+-----+-----+----+-----+----+-----+
|letter|xbox|ybox|width|high|onpix|xbar|ybar|x2bar|y2bar|xybar|x2ybr|xy2br|xege|xegvy|yege|yegvx|
+------+----+----+-----+----+-----+----+----+-----+-----+-----+-----+-----+----+-----+----+-----+
|     T|   2|   8|    3|   5|    1|   8|  13|    0|    6|    6|   10|    8|   0|    8|   0|    8|
|     I|   5|  12|    3|   7|    2|  10|   5|    5|    4|   13|    3|    9|   2|    8|   4|   10|
|     D|   4|  11|    6|   8|    6|  10|   6|    2|    6|   10|    3|    7|   3|    7|   3|    9|
|     N|   7|  11|    6|   6|    3|   5|   9|    4|    6|    4|    4|   10|   6|   10|   2|    8|
|     G|   2|   1|    3|   1|    1|   8|   6|    6|    6|    6|    5|    9|   1|    7|   5|   10|
|     S|   4|  11|    5|   8|    3|   8|   8|    6|    9|    5|    6|    6|   0|    8|   9|    7|
|     B|   4|   2|    5|   4|    4|   8|   7|    6|    6|    7|    6|    6|   2|    8|   7|   10|
|     A|   1|   1|  

In [ ]:
#dependent variables--letter

In [39]:
data.describe()

DataFrame[summary: string, letter: string, xbox: string, ybox: string, width: string, high: string, onpix: string, xbar: string, ybar: string, x2bar: string, y2bar: string, xybar: string, x2ybr: string, xy2br: string, xege: string, xegvy: string, yege: string, yegvx: string]

In [40]:
data.dtypes

[('letter', 'string'),
 ('xbox', 'int'),
 ('ybox', 'int'),
 ('width', 'int'),
 ('high', 'int'),
 ('onpix', 'int'),
 ('xbar', 'int'),
 ('ybar', 'int'),
 ('x2bar', 'int'),
 ('y2bar', 'int'),
 ('xybar', 'int'),
 ('x2ybr', 'int'),
 ('xy2br', 'int'),
 ('xege', 'int'),
 ('xegvy', 'int'),
 ('yege', 'int'),
 ('yegvx', 'int')]

In [41]:
data.columns

['letter',
 'xbox',
 'ybox',
 'width',
 'high',
 'onpix',
 'xbar',
 'ybar',
 'x2bar',
 'y2bar',
 'xybar',
 'x2ybr',
 'xy2br',
 'xege',
 'xegvy',
 'yege',
 'yegvx']

In [60]:
data.schema

StructType(List(StructField(letter,StringType,true),StructField(xbox,IntegerType,true),StructField(ybox,IntegerType,true),StructField(width,IntegerType,true),StructField(high,IntegerType,true),StructField(onpix,IntegerType,true),StructField(xbar,IntegerType,true),StructField(ybar,IntegerType,true),StructField(x2bar,IntegerType,true),StructField(y2bar,IntegerType,true),StructField(xybar,IntegerType,true),StructField(x2ybr,IntegerType,true),StructField(xy2br,IntegerType,true),StructField(xege,IntegerType,true),StructField(xegvy,IntegerType,true),StructField(yege,IntegerType,true),StructField(yegvx,IntegerType,true)))

In [50]:
from pyspark.ml.feature import StringIndexer,VectorAssembler 

In [58]:
str_idx=StringIndexer(inputCol='letter',outputCol='new_letter')

In [66]:
vect_assm=VectorAssembler(inputCols=['xbox','ybox','width','high','onpix','xbar','ybar','x2bar','y2bar','xybar','x2ybr','xy2br','xege','xegvy','yege','yegvx'],outputCol="all_feature")

In [61]:
from pyspark.ml.classification import DecisionTreeClassifier

In [67]:
tree=DecisionTreeClassifier(featuresCol="all_feature",labelCol="new_letter")

In [68]:
from pyspark.ml import Pipeline
mypipeline=Pipeline(stages=[str_idx,vect_assm,tree])

In [69]:
training,test=data.randomSplit([0.75,0.25])

In [70]:
fittree=mypipeline.fit(training)

In [ ]:
#renaming of the column
#data= data.withColumnRenamed("column_name","new column name")

In [72]:
result=fittree.transform(test)
result.show(5)

+------+----+----+-----+----+-----+----+----+-----+-----+-----+-----+-----+----+-----+----+-----+----------+--------------------+--------------------+--------------------+----------+
|letter|xbox|ybox|width|high|onpix|xbar|ybar|x2bar|y2bar|xybar|x2ybr|xy2br|xege|xegvy|yege|yegvx|new_letter|         all_feature|       rawPrediction|         probability|prediction|
+------+----+----+-----+----+-----+----+----+-----+-----+-----+-----+-----+----+-----+----+-----+----------+--------------------+--------------------+--------------------+----------+
|     A|   1|   0|    2|   0|    0|   7|   4|    2|    0|    7|    2|    8|   2|    7|   1|    8|       2.0|[1.0,0.0,2.0,0.0,...|[0.0,0.0,461.0,0....|[0.0,0.0,0.997835...|       2.0|
|     A|   1|   0|    2|   0|    0|   7|   4|    2|    0|    7|    2|    8|   2|    7|   1|    8|       2.0|[1.0,0.0,2.0,0.0,...|[0.0,0.0,461.0,0....|[0.0,0.0,0.997835...|       2.0|
|     A|   1|   0|    2|   0|    0|   8|   4|    2|    0|    7|    2|    8|   2|    7

In [74]:
#
MulticlassClassificationEvaluator()

MulticlassClassificationEvaluator_5c5c42133530

In [73]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
eval_obj=MulticlassClassificationEvaluator(predictionCol="prediction",labelCol="new_letter")
res=eval_obj.evaluate(result)
print(res)

0.3233598184407316


In [75]:
eval_obj=MulticlassClassificationEvaluator(predictionCol="prediction",labelCol="new_letter",metricName="accuracy")
res=eval_obj.evaluate(result)
print(res)

0.36788899900891975


In [77]:
eval_obj=MulticlassClassificationEvaluator(predictionCol="prediction",labelCol="new_letter",metricName="weightedPrecision")
res=eval_obj.evaluate(result)
print(res)

0.4137523144014273


In [79]:
eval_obj=MulticlassClassificationEvaluator(predictionCol="prediction",labelCol="new_letter",metricName="weightedRecall")
res=eval_obj.evaluate(result)
print(res)

0.3678889990089197


In [80]:
metriclist=["f1","accuracy","weightedPrecision","weightedRecall","weightedTruePositiveRate", "weightedFalsePositiveRate","weightedFMeasure","truePositiveRateByLabel","falsePositiveRateByLabel","precisionByLabel","recallByLabel","fMeasureByLabel","logLoss","hammingLoss"]
for metric in metriclist:
  eval_obj=MulticlassClassificationEvaluator(predictionCol="prediction",labelCol="new_letter",metricName=metric)
  print(metric,":",eval_obj.evaluate(result))



f1 : 0.3233598184407316
accuracy : 0.36788899900891975
weightedPrecision : 0.4137523144014273
weightedRecall : 0.3678889990089197
weightedTruePositiveRate : 0.3678889990089197
weightedFalsePositiveRate : 0.025874809919870873
weightedFMeasure : 0.3233598184407316
truePositiveRateByLabel : 0.8325123152709359
falsePositiveRateByLabel : 0.1493184634448575
precisionByLabel : 0.18946188340807174
recallByLabel : 0.8325123152709359
fMeasureByLabel : 0.308675799086758
logLoss : 2.0660805099632853
hammingLoss : 0.6321110009910803


In [ ]:
07/05/2022

# Regression problem

In [81]:
df=session.read.csv("energy_cooling.csv",header=True,inferSchema=True)

In [82]:
df.show(5)

+----+-----+-----+------+---+---+---+---+-----+
|  X1|   X2|   X3|    X4| X5| X6| X7| X8|   Y2|
+----+-----+-----+------+---+---+---+---+-----+
|0.98|514.5|294.0|110.25|7.0|  2|0.0|  0|21.33|
|0.98|514.5|294.0|110.25|7.0|  3|0.0|  0|21.33|
|0.98|514.5|294.0|110.25|7.0|  4|0.0|  0|21.33|
|0.98|514.5|294.0|110.25|7.0|  5|0.0|  0|21.33|
| 0.9|563.5|318.5| 122.5|7.0|  2|0.0|  0|28.28|
+----+-----+-----+------+---+---+---+---+-----+
only showing top 5 rows



In [85]:
df.dtypes

[('X1', 'double'),
 ('X2', 'double'),
 ('X3', 'double'),
 ('X4', 'double'),
 ('X5', 'double'),
 ('X6', 'int'),
 ('X7', 'double'),
 ('X8', 'int'),
 ('Y2', 'double')]

In [86]:
df.describe()

DataFrame[summary: string, X1: string, X2: string, X3: string, X4: string, X5: string, X6: string, X7: string, X8: string, Y2: string]

In [87]:
df.printSchema()

root
 |-- X1: double (nullable = true)
 |-- X2: double (nullable = true)
 |-- X3: double (nullable = true)
 |-- X4: double (nullable = true)
 |-- X5: double (nullable = true)
 |-- X6: integer (nullable = true)
 |-- X7: double (nullable = true)
 |-- X8: integer (nullable = true)
 |-- Y2: double (nullable = true)



In [93]:
from pyspark.ml.feature import VectorAssembler
vect=VectorAssembler(inputCols=['X1','X2','X3','X4','X5','X6','X7','X8'],outputCol="all_feature")

In [94]:
from pyspark.ml.regression import LinearRegression
model=LinearRegression(featuresCol="all_feature",labelCol="Y2")

In [95]:
from pyspark.ml import Pipeline
mypipeline2=Pipeline(stages=[vect,model])

In [96]:
training,test=df.randomSplit([0.8,0.2])
fitmodel=mypipeline2.fit(training)

In [97]:
result=fitmodel.transform(test)
result.show(5)

+----+-----+-----+-----+---+---+----+---+-----+--------------------+------------------+
|  X1|   X2|   X3|   X4| X5| X6|  X7| X8|   Y2|         all_feature|        prediction|
+----+-----+-----+-----+---+---+----+---+-----+--------------------+------------------+
|0.62|808.5|367.5|220.5|3.5|  2| 0.1|  1|14.34|[0.62,808.5,367.5...| 15.25424748160792|
|0.62|808.5|367.5|220.5|3.5|  2| 0.1|  4|14.37|[0.62,808.5,367.5...|15.317033907897141|
|0.62|808.5|367.5|220.5|3.5|  3| 0.1|  5|13.97|[0.62,808.5,367.5...|15.368268629062797|
|0.62|808.5|367.5|220.5|3.5|  3|0.25|  1|15.95|[0.62,808.5,367.5...| 17.45194716470057|
|0.62|808.5|367.5|220.5|3.5|  3|0.25|  3|14.96|[0.62,808.5,367.5...|17.493804782226718|
+----+-----+-----+-----+---+---+----+---+-----+--------------------+------------------+
only showing top 5 rows



In [99]:
from pyspark.ml.evaluation import RegressionEvaluator
reg_eval=RegressionEvaluator(predictionCol="prediction",labelCol="Y2")
accuracy=reg_eval.evaluate(result)
print(accuracy)

3.387097175445811


In [100]:
metric_l=["rmse","mse","mae","r2"]
for metric in metric_l:
  reg_eval=RegressionEvaluator(predictionCol="prediction",labelCol="Y2",metricName=metric)
  accuracy=reg_eval.evaluate(result)
  print(metric,":",accuracy)



rmse : 3.387097175445811
mse : 11.472427275912992
mae : 2.3445397488875526
r2 : 0.8740290571136552
